# BIZ 프로젝트 : 부실기업 예측

## Step1 : 데이터 불러오기

외부감사를 받아야 하는 법인을 의미하는 외부감사기업을 대상으로    
20-24년의 기간의 재무비율 115개의 항목에 대해서 ValueSearch를 이용하여 데이터를 수집하였다.  

https://www.nicevse.com/vse/main.html

### 필요라이브러리 불러오기

In [23]:
import pandas as pd

### 데이터 불러오기

In [24]:
RANDOM_STATE = 110

# Excel 파일 읽기
data = pd.read_excel("../../project/data/financial_data.xlsx", header=1, sheet_name='Sheet1')

In [25]:
len(data)

39861

외부감사기업으로 약 4만개의 기업의 데이터를 확보하였다.

In [26]:
data.columns

Index(['업체코드', '종목코드', '종목명', '_0', '_0.1', '_0.2', '_0.3', '_0.4',
       '2020/Annual\n191000.<성장성에 관한 지표>', '2021/Annual\n191000.<성장성에 관한 지표>',
       ...
       '2020/Annual\n196050.조세공과(구성비)', '2021/Annual\n196050.조세공과(구성비)',
       '2022/Annual\n196050.조세공과(구성비)', '2023/Annual\n196050.조세공과(구성비)',
       '2024/Annual\n196050.조세공과(구성비)', '2020/Annual\n196060.감가상각비(구성비)',
       '2021/Annual\n196060.감가상각비(구성비)', '2022/Annual\n196060.감가상각비(구성비)',
       '2023/Annual\n196060.감가상각비(구성비)', '2024/Annual\n196060.감가상각비(구성비)'],
      dtype='object', length=578)

In [27]:
# 확인할 변수 목록
variables_to_check = ['_0', '_0.1', '_0.2', '_0.3', '_0.4']

# 각 변수의 고유 값 확인
for column in variables_to_check:
    if column in data.columns:
        unique_values = data[column].unique()
        print(f"'{column}' 변수의 고유 값: {unique_values}")
    else:
        print(f"'{column}' 변수는 데이터프레임에 존재하지 않습니다.")

'_0' 변수의 고유 값: [nan]
'_0.1' 변수의 고유 값: [nan]
'_0.2' 변수의 고유 값: [nan]
'_0.3' 변수의 고유 값: [nan]
'_0.4' 변수의 고유 값: [nan]


In [28]:
# 드랍할 변수 목록
variables_to_drop = ['_0', '_0.1', '_0.2', '_0.3', '_0.4']

# 변수 드랍
df = data.drop(columns=variables_to_drop)

데이터를 수집하는 과정에서 불필요하다고 생각이 들은 변수들에 대해서 결측값을 확인하였고    
['_0', '_0.1', '_0.2', '_0.3', '_0.4'] 5개의 변수 모두 결측으로 존재하여 제거해 주었다.  

In [29]:
df.columns

Index(['업체코드', '종목코드', '종목명', '2020/Annual\n191000.<성장성에 관한 지표>',
       '2021/Annual\n191000.<성장성에 관한 지표>', '2022/Annual\n191000.<성장성에 관한 지표>',
       '2023/Annual\n191000.<성장성에 관한 지표>', '2024/Annual\n191000.<성장성에 관한 지표>',
       '2020/Annual\n191010.총자산증가율', '2021/Annual\n191010.총자산증가율',
       ...
       '2020/Annual\n196050.조세공과(구성비)', '2021/Annual\n196050.조세공과(구성비)',
       '2022/Annual\n196050.조세공과(구성비)', '2023/Annual\n196050.조세공과(구성비)',
       '2024/Annual\n196050.조세공과(구성비)', '2020/Annual\n196060.감가상각비(구성비)',
       '2021/Annual\n196060.감가상각비(구성비)', '2022/Annual\n196060.감가상각비(구성비)',
       '2023/Annual\n196060.감가상각비(구성비)', '2024/Annual\n196060.감가상각비(구성비)'],
      dtype='object', length=573)

기업의 인덱스와 관련된 변수 ['업체코드', '종목코드', '종목명']를 제외하면 변수는 총 570개  
570 -> 5(연도) * 114(재무비율) 으로 재무비율 변수로 총 114개  

In [30]:
# 열 이름 변환 함수
def transform_column_name(col_name):
    if col_name in ['업체코드', '종목코드', '종목명']:
        return col_name
    parts = col_name.split('\n')
    if len(parts) == 2:
        year, desc = parts
        desc = desc.split('.')[1]  # 변수 키워드 부분만 추출
        return f"{year.split('/')[0]}/{desc}"
    return col_name

# 열 이름 변환 적용
df.columns = [transform_column_name(col) for col in df.columns]

# 변환된 열 이름 출력
print(df.columns)

Index(['업체코드', '종목코드', '종목명', '2020/<성장성에 관한 지표>', '2021/<성장성에 관한 지표>',
       '2022/<성장성에 관한 지표>', '2023/<성장성에 관한 지표>', '2024/<성장성에 관한 지표>',
       '2020/총자산증가율', '2021/총자산증가율',
       ...
       '2020/조세공과(구성비)', '2021/조세공과(구성비)', '2022/조세공과(구성비)', '2023/조세공과(구성비)',
       '2024/조세공과(구성비)', '2020/감가상각비(구성비)', '2021/감가상각비(구성비)',
       '2022/감가상각비(구성비)', '2023/감가상각비(구성비)', '2024/감가상각비(구성비)'],
      dtype='object', length=573)


변수명의 주요 키워드만을 이용하여 새로운 변수명으로 변환   
ex) 2020/Annual\n196050.조세공과(구성비) -> 2020/조세공과(구성비)   

In [33]:
import pandas as pd

# 열 이름 중복 확인 함수
def check_duplicate_columns(df):
    duplicates = df.columns[df.columns.duplicated()]
    if len(duplicates) > 0:
        print("중복되는 열 이름:", duplicates)
    else:
        print("중복되는 열 이름이 없습니다.")

# 중복 확인 함수 호출
check_duplicate_columns(df)

중복되는 열 이름이 없습니다.


변환된 변수명간에 중복되는 경우를 확인하였고 없음을 확인하였다.

In [36]:
len(df)

39861

In [ ]:
# 기업의 인덱스 관련 변수 지정하여 중복 확인
df[['업체코드', '종목코드', '종목명']].value_counts()

업체코드     종목코드     종목명      
N000292  A059210  메타바이오메드      1
N813419  A021880  메이슨캐피탈       1
N813605  A022100  포스코DX        1
N813796  A033650  메타넷디지털(주)    1
N813826  A023780  신한캐피탈(주)     1
                              ..
N290082  A117560  (주)삼우아이엠씨    1
N290141  A214870  한울BnC        1
N290171  A317120  라닉스          1
N290565  A305090  마이크로디지탈      1
NT42259  A456190  큐라켐          1
Length: 3670, dtype: int64

In [35]:
data.drop_duplicates(subset=['종목명'], keep='first', inplace=True)

In [36]:
data['종목명'].value_counts()

종목명
(주)신한은행         1
(주)포트디자인랩       1
(주)영신하우징        1
(주)아이브이신라       1
농업회사법인(주)무의     1
               ..
제트에프렘페더샤시(주)    1
(주)마창대교         1
(주)삼희종합건설       1
(주)포럼씨앤디        1
(주)엘림글로벌        1
Name: count, Length: 38798, dtype: int64